<a href="https://colab.research.google.com/github/patil-suraj/question_generation/blob/master/question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers==3.0.0

Requirement already up-to-date: transformers==3.0.0 in /usr/local/lib/python3.6/dist-packages (3.0.0)


In [ ]:
!python -m nltk.downloader punkt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip3 install nlp

In [ ]:
!pip3 install git+https://github.com/Maluuba/nlg-eval.git@master


  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-req-build-f7gz4_d0
  Running command git clone -q https://github.com/Maluuba/nlg-eval.git /tmp/pip-req-build-f7gz4_d0
  Created wheel for nlg-eval: filename=nlg_eval-2.3-cp36-none-any.whl size=68175138 sha256=5c2f4859d104a6170797a91c7fbfa070f1d256ad6ee817abb6b7680a20cbfc43
  Stored in directory: /tmp/pip-ephem-wheel-cache-ziur56iu/wheels/a5/7c/fd/f312beca2adcc3f49cb40570730658dad37bb5709f5d237a56
Successfully built nlg-eval


In [ ]:
!nlg-eval --setup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Installing to /root/.cache/nlgeval
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /root/.cache/nlgeval' again.
glove2word2vec.py: 100% 1.00/1.00 [00:00<00:00, 531 chunks/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My\ Drive/question_generation/question_generation/

/content/drive/My Drive/question_generation/question_generation


In [ ]:
ls

data/                            README.md
data_collator.py                 run_qg.py
eval.py                          runs/
hypothesis_t5-base-qg-hl_tw.txt  t5_qg_tokenizer/
hypothesis_t5-base-qg-hl.txt     t5-small-qg-hl/
LICENSE                          t5-small-qg-hl-15/
notebooks/                       t5-small-qg-hl-15-log/
pipelines.py                     t5-small-qg-hl-15-old/
prepare_data.py                  trainer.py
__pycache__/                     tweetqadata/
question_generation.ipynb        utils.py


In [ ]:
# Prepare TweetQA data 
!python3 prepare_data.py --task qg --model_type t5 --dataset_path data/tweet_multitask --qg_format highlight_qg_format --max_source_length 512 --max_target_length 32 --train_file_name train_data_qg_hl_t5_tweet_automatic.pt  --valid_file_name valid_data_qg_hl_t5_tweet_automatic.pt

In [ ]:
%%capture loggingfine
!python3 run_qg.py \
    --model_name_or_path t5-small \
    --model_type t5 \
    --tokenizer_name_or_path t5_qg_tokenizer \
    --output_dir t5-small-qg-hl-15 \
    --train_file_path data/train_data_qg_hl_t5_tweet_automatic.pt \
    --valid_file_path data/valid_data_qg_hl_t5_tweet_automatic.pt \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --gradient_accumulation_steps 8 \
    --learning_rate 1e-4 \
    --num_train_epochs 15 \
    --seed 42 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --logging_steps 100\
    --logging_dir t5-small-qg-hl-15-log\
    --overwrite_output_dir

In [ ]:
# original with 2 epochs of training
#!python3 run_qg.py \
#    --model_name_or_path t5-small \
#    --model_type t5 \
#    --tokenizer_name_or_path t5_qg_tokenizer \
#    --output_dir t5-small-qg-hl \
#    --train_file_path data/train_data_qg_hl_t5.pt \
#    --valid_file_path data/valid_data_qg_hl_t5.pt \
#    --per_device_train_batch_size 16 \
#    --per_device_eval_batch_size 16 \
#    --gradient_accumulation_steps 8 \
#    --learning_rate 1e-4 \
#    --num_train_epochs 2 \
#    --seed 42 \
#    --do_train \
#    --do_eval \
#    --evaluate_during_training \
#    --logging_steps 100

In [ ]:
#using originally pretrained model valhalla
!python eval.py \
    --model_name_or_path  valhalla/t5-small-qa-qg-hl \
    --valid_file_path data/valid_data_qg_hl_t5_tweet_automatic.pt \
    --model_type t5 \
    --num_beams 4 \
    --max_decoding_length 32 \
    --output_path hypothesis_t5-base-qg-hl_tweet_atomatic_valhalla_model.txt

2020-08-15 18:11:08.579510: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
  0% 0/34 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)
100% 34/34 [00:38<00:00,  1.14s/it]


In [ ]:
#using finetuned  model with 15 epochs
!python eval.py \
    --model_name_or_path  t5-small-qg-hl-15-old \
    --valid_file_path data/valid_data_qg_hl_t5_tweet_automatic.pt \
    --model_type t5 \
    --num_beams 4 \
    --max_decoding_length 32 \
    --output_path hypothesis_t5-base-qg-hl_t5_tweet_automatic.txt

2020-08-15 18:09:09.211005: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
  0% 0/34 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)
100% 34/34 [00:35<00:00,  1.05s/it]


In [21]:
#evaluate valhalla model
!nlg-eval --hypothesis=hypothesis_t5-base-qg-hl_tweet_atomatic_valhalla_model.txt --references=data/tweet_dev_automatic_reference.txt --no-skipthoughts --no-glove 

Using data from /root/.cache/nlgeval
In case of broken downloads, remove the directory and run setup again.
Bleu_1: 0.166936
Bleu_2: 0.102170
Bleu_3: 0.069558
Bleu_4: 0.048868
METEOR: 0.198409
ROUGE_L: 0.179716
CIDEr: 0.549717


In [20]:
#evaluate fine-tuned model
!nlg-eval --hypothesis=hypothesis_t5-base-qg-hl_t5_tweet_automatic.txt --references=data/tweet_dev_automatic_reference.txt --no-skipthoughts --no-glove 

Using data from /root/.cache/nlgeval
In case of broken downloads, remove the directory and run setup again.
Bleu_1: 0.323073
Bleu_2: 0.221209
Bleu_3: 0.161737
Bleu_4: 0.122189
METEOR: 0.209194
ROUGE_L: 0.345015
CIDEr: 1.304266


In [19]:
ls data

references_tw.txt                       tweet_dev_automatic_reference.txt
references.txt                          tweet_multitask/
squad_multitask/                        tweet_multitask_old/
train_data_qg_hl_t5_sq.pt               valid_data_qg_hl_t5_sq.pt
train_data_qg_hl_t5_tweet_automatic.pt  valid_data_qg_hl_t5_tweet_automatic.pt
train_data_qg_hl_t5_tw.pt               valid_data_qg_hl_t5_tw.pt
